In [1]:
import boto3
ec2 = boto3.client('ec2')
ssm = boto3.client('ssm')
elb = boto3.client('elbv2')

### [1] Create 2 EC2 instances in two different availability zones of a specific region. 

In [2]:
KEY_NAME = "22962256-key"
SG_NAME = "22962256-sg"
# Prepare subnet for ec2 creation
VPC_ID = "vpc-00da1b229d10a51b6"
SUBNETS = [
    {"id": "subnet-0a7d8e51199753df1", "name": "ap-southeast-2a"},
    {"id": "subnet-0c1878c6a739707b7", "name": "ap-southeast-2b"},
]

In [ ]:
# Create Key Pair
key_response = ec2.create_key_pair(
    KeyName=KEY_NAME
)

with open("22962256.pem", "wb") as file:
    # Write the variable to the file
    file.write(key_response['KeyMaterial'].encode())

In [5]:
# Create Security Group
sg_response = ec2.create_security_group(
    Description="security group for development environment by boto3",
    GroupName=SG_NAME,
    VpcId=VPC_ID,
)

sg_id = sg_response["GroupId"]

ec2.authorize_security_group_ingress(
    GroupId=sg_id,
    IpPermissions=[
        {
            "IpProtocol": "tcp",
            "FromPort": 22,
            "ToPort": 22,
            "IpRanges": [{"CidrIp": "0.0.0.0/0"}],
        },
        {
            "IpProtocol": "tcp",
            "FromPort": 80,
            "ToPort": 80,
            "IpRanges": [{"CidrIp": "0.0.0.0/0"}],
        },
    ],
)

{'Return': True,
 'SecurityGroupRules': [{'SecurityGroupRuleId': 'sgr-07cb1274d26043234',
   'GroupId': 'sg-069f2ce852b14278a',
   'GroupOwnerId': '489389878001',
   'IsEgress': False,
   'IpProtocol': 'tcp',
   'FromPort': 22,
   'ToPort': 22,
   'CidrIpv4': '0.0.0.0/0'},
  {'SecurityGroupRuleId': 'sgr-09f97549385bf5d1f',
   'GroupId': 'sg-069f2ce852b14278a',
   'GroupOwnerId': '489389878001',
   'IsEgress': False,
   'IpProtocol': 'tcp',
   'FromPort': 80,
   'ToPort': 80,
   'CidrIpv4': '0.0.0.0/0'}],
 'ResponseMetadata': {'RequestId': '04f0b74b-7cee-4c6c-818f-55813ced5418',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '04f0b74b-7cee-4c6c-818f-55813ced5418',
   'cache-control': 'no-cache, no-store',
   'strict-transport-security': 'max-age=31536000; includeSubDomains',
   'content-type': 'text/xml;charset=UTF-8',
   'content-length': '1124',
   'date': 'Sat, 23 Sep 2023 07:07:10 GMT',
   'server': 'AmazonEC2'},
  'RetryAttempts': 0}}

In the following step, we launch EC2 instances and install Apache web server on them. We will use these instances as targets for our load balancer.
[Run commands on your Linux instance at launch](https://docs.aws.amazon.com/AWSEC2/latest/UserGuide/user-data.html)

In [12]:
def get_userdata(name):
    return (
        """
            #cloud-config
            runcmd:
            - apt-get update
            - apt install apache2 -y
            - chmod 777 /var/www/html/index.html
            - echo "This is page from 22962256-%s." > /var/www/html/index.html
        """
        % name
    )


instance_responces = list(
    map(
        lambda subnet: ec2.run_instances(
            ImageId="ami-d38a4ab1",
            InstanceType="t2.micro",
            MaxCount=1,
            MinCount=1,
            KeyName=KEY_NAME,
            SubnetId=subnet["id"],
            SecurityGroupIds=[sg_id],
            UserData=get_userdata(subnet["name"]),
            TagSpecifications=[
                {
                    "ResourceType": "instance",
                    "Tags": [
                        {"Key": "Name", "Value": "22962256-" + subnet["name"]},
                    ],
                },
            ],
        ),
        SUBNETS,
    )
)

In [7]:
instance_ids = list(
    map(lambda response: response["Instances"][0]["InstanceId"], instance_responces)
)

### Clean-up

In [ ]:
ec2.delete_key_pair(KeyName=KEY_NAME)
ec2.delete_security_group(GroupId=sg_id)
ec2.terminate_instances(InstanceIds=instance_ids)